Simple RAG  USing our Custom Embedding Class
USed openserach as vector DB.
Tried with different Chain mathod like. Stuff and MapReduce

Importing Required Loader from langchain

In [ ]:
from langchain.document_loaders import (
    UnstructuredURLLoader,
    UnstructuredWordDocumentLoader,
    ConfluenceLoader
)

In [ ]:
# Loading Data from URL uding URLLoader
loaderurl = UnstructuredURLLoader(
    urls = [
        "https://www.who.int/westernpacific/news-room/feature-stories/item/4-things-to-know-about-covid-19-treatment-in-the-pacific"
    ]
)
url_data = loaderurl.load()
url_data

In [ ]:
# Loading Data from Docs uding DocsLoader
file = "./indian.docx"
loaderdocs = UnstructuredWordDocumentLoader(file)
doc_data = loaderdocs.load()
doc_data

In [ ]:
# Loading Data from Confluence
loader =ConfluenceLoader(
    url="https://adityas181.atlassian.net/wiki", username="adityas181@gmail.com", api_key="ATATT3xFfGF0enLfG-S2di7ACvuvLpCQXA6fkuyK2ZiHp3ES2sHWsaSvpJVTsf1LPQ_YNif-RZu0A3NhyDnpVAQH3B3LiWxJNOjKHBjEzKxCgwWVeri74H5Hf_Tlpqbs3nQn-tEQesEbRN-ih55TtKmrrEsGz8Ljx3PfcjhCBsAIB8XDi9ye86s=22D8F09F"
)
confluence_data = loader.load(space_key="documentat")
confluence_data 


In [ ]:
# adding all data from different sources for spliting and ingestion
data = url_data + doc_data + confluence_data 
data

Now Spliting the all data using RecursiveCharacterTextSplitter

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(
        separators=['\n\n', '\n', '.', ','],
        chunk_size=800,
        chunk_overlap=200
    )
 
docs = text_splitter.split_documents(data)

docs

we can extract the page content and metadata from the docs

In [ ]:
page_content = [doc.page_content for doc in docs]
meta = [doc.metadata for doc in docs]
print(len(page_content))
print(len(meta))
print(page_content)
print(meta)


We loaded the data from different sources.
Now importent part is to create a ingestion Pipeline to ingest all data in vector DB
Here I am using Opensearch for as vector DB for indexing.
Created my Custom embedding class to use our embedding models. I am using openai embedding

In [ ]:
# ingestion pipeline
from embedding import MyEmbedding
from langchain.vectorstores import OpenSearchVectorSearch
embeddings = MyEmbedding()  # loading custom embedding class
index_name=''  # index name to create. it created automatically by opensearach
username = ""
password = ""
# creating object of opensearch vector
docsearch = OpenSearchVectorSearch(
    index_name=index_name,
    embedding_function=embeddings,
    opensearch_url="",
    http_auth = (username,password)
)
print(docsearch.index_name)
print(docsearch.client.info())


In [ ]:
# ingestion the Docs that loader from different sources
ingest = docsearch.add_documents(documents=docs)
print(ingest)

Similarity search

In [ ]:
# Doing similarity search on thr docs. 
# using cosinesimil serach type
query ="which are leading authorities for food and drugs?"
#query = "which are leading authorities for food and drugs?"
#query = "what are major economic sectors"
docs = docsearch.similarity_search_with_score(query, space_type="cosinesimil")
citation = [
    {
    "page_content" :doc[0].page_content,
    "score" : doc[1]
    }
    for doc in docs
]

citation

Here I am making ConversationalRetrievalChain from Langchain
Using Stuff method

In [ ]:
## Stuff method
from embedding import MyEmbedding
from langchain.chains import ConversationalRetrievalChain
from langchain.chat_models import ChatOpenAI
openai_key= ""
embeddings = MyEmbedding()
index_name=''
username =  ''
password = ''
vector_db = OpenSearchVectorSearch(
    index_name=index_name,
    embedding_function=embeddings,
    opensearch_url="",
    http_auth = (username,password),
    timeout=300,
    verbose=True    
)

chain = ConversationalRetrievalChain.from_llm(ChatOpenAI(openai_api_key=openai_key), chain_type="stuff",
                                   retriever=vector_db.as_retriever(),
                                   return_source_documents=True)
# chain = ConversationalRetrievalChain.from_llm(ChatOpenAI(openai_api_key=openai_key), chain_type="stuff",
#                                    retriever=vector_db.as_retriever(search_kwargs={"k": 3}))
chain

Asking question from document.
First it will make embedding of query and then search the embedding in opensearch at particular mebedding then it return the citation and then send citation to LLM and return the Ans of query.
HEre we are alos making chat history so it can retain the ans of prevoius question.
langchain.debug=True will show each step that langchaon do internally


In [ ]:
import langchain
query = "which are leading authorities for food and drugs?"
#query ="what are major economic sectors"
#query = "How to use expansion in the REST APIs?"

langchain.debug=True
chat_history= []
result = chain({"question": query,'chat_history': chat_history})
ans = result["answer"]
citation=result["source_documents"]
print(f" Answer ---{ans}")
print(citation)

In [ ]:
query = "what is the price of Tiago iCNG?"
#langchain.debug=True
chat_history= []
result = chain({"question": query,'chat_history': chat_history})
print(result)

In [ ]:
# here we adding the answer of prevoius query and sned angain to LLM. By this it maintain the context of prevous questions.
chat_history += [(query, result["answer"])]
query = "can you divide the range by 2"
result = chain({"question": query,'chat_history': chat_history}, return_only_outputs=True)
print(chat_history)
print(result)


Now we are using Map reduce method.
This method answer the query from accurate but it amke more call to LLM.

In [ ]:
## map reduce 
from embedding import MyEmbedding
from langchain.chains import ConversationalRetrievalChain
from langchain.chat_models import ChatOpenAI
embeddings = MyEmbedding()
index_name=''
username =  ''
password = ''
vector_db = OpenSearchVectorSearch(
    index_name=index_name,
    embedding_function=embeddings,
    opensearch_url="",
    http_auth = (username,password),
    timeout=300,
    verbose=True
    
)
chain = ConversationalRetrievalChain.from_llm(ChatOpenAI(openai_api_key=openai_key), chain_type="map_reduce",
                                   retriever=vector_db.as_retriever())
chain

In [ ]:

query = "what is the price of Tiago iCNG?"

chat_history= []
result = chain({"question": query,'chat_history': chat_history})
print(result)

In [ ]:
# here we adding the answer of prevoius query and sned angain to LLM. By this it maintain the context of prevous questions.
chat_history += [(query, result["answer"])]
query = "can you divide the range by 2"
result = chain({"question": query,'chat_history': chat_history}, return_only_outputs=True)
print(chat_history)
print(result)
